In [2]:
from google.colab import files
uploaded = files.upload()

Saving Customers.csv to Customers.csv
Saving Products.csv to Products.csv
Saving Transactions.csv to Transactions.csv


In [14]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, silhouette_score

# Load the datasets
customers_path = '/content/Customers.csv'
products_path = '/content/Products.csv'
transactions_path = '/content/Transactions.csv'

In [12]:
import pandas as pd

# Load the datasets
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

# Display basic information about each dataset
customers_info = customers.info()
products_info = products.info()
transactions_info = transactions.info()

# Display the first few rows of each dataset
customers_head = customers.head()
products_head = products.head()
transactions_head = transactions.head()

customers_info, customers_head, products_info, products_head, transactions_info, transactions_head


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    100 non-null    object 
 1   ProductName  100 non-null    object 
 2   Category     100 non-null    object 
 3   Price        100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  


(None,
   CustomerID        CustomerName         Region  SignupDate
 0      C0001    Lawrence Carroll  South America  2022-07-10
 1      C0002      Elizabeth Lutz           Asia  2022-02-13
 2      C0003      Michael Rivera  South America  2024-03-07
 3      C0004  Kathleen Rodriguez  South America  2022-10-09
 4      C0005         Laura Weber           Asia  2022-08-15,
 None,
   ProductID              ProductName     Category   Price
 0      P001     ActiveWear Biography        Books  169.30
 1      P002    ActiveWear Smartwatch  Electronics  346.30
 2      P003  ComfortLiving Biography        Books   44.12
 3      P004            BookWorld Rug   Home Decor   95.69
 4      P005          TechPro T-Shirt     Clothing  429.31,
 None,
   TransactionID CustomerID ProductID      TransactionDate  Quantity  \
 0        T00001      C0199      P067  2024-08-25 12:38:23         1   
 1        T00112      C0146      P067  2024-05-27 22:23:54         1   
 2        T00166      C0127      P067  20

In [13]:
# Merge datasets to create a unified dataset for clustering

# Merge Customers and Transactions based on CustomerID
customer_transactions = transactions.merge(customers, on='CustomerID', how='inner')

# Merge the resulting dataset with Products based on ProductID
unified_data = customer_transactions.merge(products, on='ProductID', how='inner')

# Aggregating data for each customer
customer_features = unified_data.groupby('CustomerID').agg(
    total_spending=('TotalValue', 'sum'),
    total_transactions=('TransactionID', 'count'),
    avg_transaction_value=('TotalValue', 'mean'),
    avg_quantity=('Quantity', 'mean'),
    unique_products=('ProductID', 'nunique'),
    regions=('Region', 'first')
).reset_index()

# Display the resulting dataset
customer_features.head()

,CustomerID,total_spending,total_transactions,avg_transaction_value,avg_quantity,unique_products,regions
0,C0001,3354.52,5,670.904,2.400000,5,South America
1,C0002,1862.74,4,465.685,2.500000,4,Asia
2,C0003,2725.38,4,681.345,3.500000,4,South America
3,C0004,5354.88,8,669.360,2.875000,8,South America
4,C0005,2034.24,3,678.080,2.333333,3,Asia


In [15]:
# Encode categorical 'regions' column
label_encoder = LabelEncoder()
customer_features['regions_encoded'] = label_encoder.fit_transform(customer_features['regions'])

# Select features for clustering
features = customer_features[['total_spending', 'total_transactions',
                               'avg_transaction_value', 'avg_quantity',
                               'unique_products', 'regions_encoded']]

# Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Apply KMeans clustering (initially testing with 3 clusters)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(scaled_features)

# Add cluster labels to the dataset
customer_features['cluster'] = cluster_labels

# Calculate evaluation metrics
db_index = davies_bouldin_score(scaled_features, cluster_labels)
silhouette_avg = silhouette_score(scaled_features, cluster_labels)

db_index, silhouette_avg, customer_features[['CustomerID', 'cluster']].head()


(1.4223538866369634,
 0.22269376425451448,
   CustomerID  cluster
 0      C0001        1
 1      C0002        1
 2      C0003        0
 3      C0004        2
 4      C0005        1)